## Evaluation
Submissions are scored on the root mean squared error.

## References
https://www.kaggle.com/code/sebastianvangerwen/1st-place-solution-tps-jun-denoising-ae issued by **@SEBASTIAN VAN GERWEN**<br>
https://towardsdatascience.com/denoising-autoencoders-dae-how-to-use-neural-networks-to-clean-up-your-data-cd9c19bc6915 issued by **@Saul Dobilas**

## Blue Print

## 0. Import Packages

In [1]:
import math
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import torch
import torch.nn.functional as F
import torch.utils.data
from torch import nn

from tqdm import tqdm

/Users/lucy/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## 1. Data Loading

In [2]:
# Load dataset
data = pd.read_csv('data.csv')
data.shape

(1000000, 81)

In [3]:
# Check data types and missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 81 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   row_id  1000000 non-null  int64  
 1   F_1_0   981603 non-null   float64
 2   F_1_1   981784 non-null   float64
 3   F_1_2   981992 non-null   float64
 4   F_1_3   981750 non-null   float64
 5   F_1_4   981678 non-null   float64
 6   F_1_5   981911 non-null   float64
 7   F_1_6   981867 non-null   float64
 8   F_1_7   981872 non-null   float64
 9   F_1_8   981838 non-null   float64
 10  F_1_9   981751 non-null   float64
 11  F_1_10  982039 non-null   float64
 12  F_1_11  981830 non-null   float64
 13  F_1_12  981797 non-null   float64
 14  F_1_13  981602 non-null   float64
 15  F_1_14  981961 non-null   float64
 16  F_2_0   1000000 non-null  int64  
 17  F_2_1   1000000 non-null  int64  
 18  F_2_2   1000000 non-null  int64  
 19  F_2_3   1000000 non-null  int64  
 20  F_2_4   1000000 non-null 

**Comments**: Column `F_1_0` ~ `F_1_14`, `F_3_0` ~ `F_3_24`, `F_4_0` ~ `F_4_14` have missing values. The types of missing values are all floats.

In [4]:
# List of features
features = data.columns.drop('row_id').tolist()
features

['F_1_0',
 'F_1_1',
 'F_1_2',
 'F_1_3',
 'F_1_4',
 'F_1_5',
 'F_1_6',
 'F_1_7',
 'F_1_8',
 'F_1_9',
 'F_1_10',
 'F_1_11',
 'F_1_12',
 'F_1_13',
 'F_1_14',
 'F_2_0',
 'F_2_1',
 'F_2_2',
 'F_2_3',
 'F_2_4',
 'F_2_5',
 'F_2_6',
 'F_2_7',
 'F_2_8',
 'F_2_9',
 'F_2_10',
 'F_2_11',
 'F_2_12',
 'F_2_13',
 'F_2_14',
 'F_2_15',
 'F_2_16',
 'F_2_17',
 'F_2_18',
 'F_2_19',
 'F_2_20',
 'F_2_21',
 'F_2_22',
 'F_2_23',
 'F_2_24',
 'F_3_0',
 'F_3_1',
 'F_3_2',
 'F_3_3',
 'F_3_4',
 'F_3_5',
 'F_3_6',
 'F_3_7',
 'F_3_8',
 'F_3_9',
 'F_3_10',
 'F_3_11',
 'F_3_12',
 'F_3_13',
 'F_3_14',
 'F_3_15',
 'F_3_16',
 'F_3_17',
 'F_3_18',
 'F_3_19',
 'F_3_20',
 'F_3_21',
 'F_3_22',
 'F_3_23',
 'F_3_24',
 'F_4_0',
 'F_4_1',
 'F_4_2',
 'F_4_3',
 'F_4_4',
 'F_4_5',
 'F_4_6',
 'F_4_7',
 'F_4_8',
 'F_4_9',
 'F_4_10',
 'F_4_11',
 'F_4_12',
 'F_4_13',
 'F_4_14']

In [5]:
f1_col, f2_col, f3_col, f4_col = [], [], [], []
for f in features:
    if f[:3] == 'F_1':
        f1_col.append(f)
    elif f[:3] == 'F_2':
        f2_col.append(f)
    elif f[:3] == 'F_3':
        f3_col.append(f)
    elif f[:3] == 'F_4':
        f4_col.append(f)

## 2. 

In [6]:
# Binomial x Random one 0 per row
def random_mask(n, k):
    mask = np.ones((n, k))

    # Set one random per row at 0
    mask[(np.arange(n), np.random.randint(0, k, n))] = 0
    
    # Add binomial probability as well
    b_mask = np.random.binomial(1, 0.5, (n, k))    # 1 trial, p=0.5
    return mask * b_mask

In [7]:
def mask_n_rows(n, k, n_missing):
    # n_missing number of indices of columns with small values
    idx = np.random.rand(n, k).argsort(1)[:, :n_missing]

    col_idx = idx.flatten()
    row_idx = np.arange(n).repeat(n_missing)
    
    mask = np.ones((n, k))
    mask[(row_idx, col_idx)] = 0
    return mask

In [8]:
# Find rows with missing values
missing_bool = data[features].isna().sum(axis=1) > 0

# Define subsets of the data with row-wise missing values
complete = data.loc[~missing_bool, features].values     # without missing values
missing = data.loc[missing_bool, features].values       # with missing values

# Split data that has no missing to use for validation set
Xtr, Xva = train_test_split(complete)

# Build training set by combining cXtr and missing data
X_mixed = np.concatenate([Xtr, missing], axis=0)

# Mask to show train values that have been imputed
srce_nan_train = np.concatenate([
    np.zeros(Xtr.shape),
    data.loc[missing_bool, features].isna().astype(np.uint8).values])

# Feature scaling
scaler = StandardScaler()
scaler.fit(data[features].values)

# Apply feature scaling
X_mixed = np.nan_to_num(scaler.transform(X_mixed), 0.0)
Xva = scaler.transform(Xva)

## 3. Build Model

In [9]:
class MLP(nn.Module):
# Dense layer with layer normalization and mish activation
    def __init__(self, input_size, output_size):
        super().__init__()
        self.dense = nn.Linear(input_size, output_size)
        self.act = nn.Mish()
        self.layernorm = nn.LayerNorm(output_size, eps=1e-6)
        
    def forward(self, x):
        x = self.dense(x)
        x = self.act(x)
        return self.layernorm(x)

In [10]:
# Msked autoencoder model
class MaskedAutoencoder(nn.Module):
    def __init__(self, n_columns, emb_dim=16,
                 units=[512, 512, 512, 512, 512, 128]):
        super().__init__()
        self.n_columns = n_columns

        # Embedding
        self.inp_proj = nn.Linear(1, emb_dim)
        self.mask_proj = nn.Linear(1, emb_dim)
        self.emb_norm = nn.LayerNorm(n_columns * emb_dim, eps=1e-6)
        
        # MLP with skip connection
        self.mlp_layers = nn.ModuleList([])
        for i in range(len(units)):
            if i==0:
                input_size = n_columns * emb_dim
            elif i==1:
                input_size = n_columns * emb_dim + units[0]
            else:
                input_size = units[i-1] + units[i-2]
            output_size = units[i]
            self.mlp_layers.append(
                MLP(input_size=input_size, output_size=output_size)
            )
                
        self.final_dense = nn.Linear(units[-1] + units[-2], self.n_columns)
        
    def forward(self, inputs:torch.Tensor, mask:torch.Tensor):
        # Embeddings
        input_embedding = self.inp_proj(torch.unsqueeze(inputs, 2))
        mask_embedding = self.mask_proj(torch.unsqueeze(1-mask, 2))
        embedding = input_embedding + mask_embedding
        embedding = torch.flatten(embedding, 1)
        x = [self.emb_norm(embedding)]
        
        # MLP
        for i in range(len(self.mlp_layers)):
            if i==0:
                z = self.mlp_layers[i](x[0])
                x.append(z)
            else:
                z = torch.cat((x[-1], x[-2]), 1)
                z = self.mlp_layers[i](z)
                x.append(z)
                
        x = torch.cat((x[-1], x[-2]), 1)
        x = self.final_dense(x)
        
        # Output modification - predict only masked values, otherwise use inputs
        outputs = torch.mul(inputs, mask) + torch.mul(1-mask, x)
        
        return outputs


In [11]:
# Helper validation method
def validate(model, valid_mask, batch_size=4096):
    assert valid_mask.shape == Xva.shape
    
    n_batches_valid = Xva.shape[0] // batch_size + 1
    
    model.eval()
    with torch.no_grad():
        ps = []
        for batch in range(n_batches_valid):
            x = torch.tensor(Xva[batch * batch_size: (batch+1) * batch_size].astype(np.float32)).to(device)
            mask = torch.tensor(valid_mask[batch * batch_size: (batch+1) * batch_size].astype(np.float32)).to(device)
            x_masked = x * mask

            p = model(x_masked, mask).cpu().numpy()
            ps.append(p)

        p = np.vstack(ps)
        mask_bool = (1 - valid_mask).astype(bool)
        rmse = np.sqrt(mean_squared_error(
            scaler.inverse_transform(p)[mask_bool],
            scaler.inverse_transform(Xva)[mask_bool]
        ))
        return rmse

In [12]:
# Loss function to mask NaNs in the original data
class MaskedMSELoss(nn.Module):
    # Mask should be 1 for masked value, 0 for unmasked value 
    def __init__(self):
        super().__init__()
        self.loss = nn.MSELoss(reduction='none')
    
    def forward(self, inputs, target, mask):
        loss = self.loss(inputs, target)
        return torch.mean(loss * (1 - mask))

In [13]:
# Defining model parameters and learning rate schedule
epochs = 300
lr_start = 0.001
lr_end = 0.00005
batch_size = 4096

# This cosine decay function is borrowed from AmbrosM in last month's TPS
def cosine_decay(epoch):
    if epochs > 1:
        w = (1 + math.cos(epoch / (epochs-1) * math.pi)) / 2
    else:
        w = 1
    return w * lr_start + (1 - w) * lr_end

In [3]:
pip install numpy --pre torch torchvision torchaudio --force-reinstall --index-url https://download.pytorch.org/whl/nightly/cu116

Looking in indexes: https://download.pytorch.org/whl/nightly/cu116
ERROR: Could not find a version that satisfies the requirement numpy (from versions: none)
ERROR: No matching distribution found for numpy
Note: you may need to restart the kernel to use updated packages.


In [4]:
torch.cuda.is_available()

False

In [15]:
# Initial weights
def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_normal_(m.weight)
        m.bias.data.fill_(0.01)

# Build model
device = 'cuda'

# Final model uses units = [2048, 2048, 2048, 1024, 512, 256, 128], but I use a smaller model for this notebook
model = MaskedAutoencoder(15, units=[512, 512, 512, 512, 512, 256, 128]).to(device)
model.apply(init_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=1)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=cosine_decay)
loss_fn = MaskedMSELoss()

AssertionError: Torch not compiled with CUDA enabled

In [24]:
# Training loop

# for epoch in epochs,
np.random.seed(6)

n = X_mixed.shape[0]
batch_size = 4096
n_batches = n // batch_size + 1
index = np.arange(n)

valid_per = 5

# Validation Mask
validation_masks = [mask_n_rows(Xva.shape, i+1) for i in range(5)]
validation_prob = list(data[f4_col].isna().sum(axis=1).value_counts() \
    / data.loc[data[f4_col].isna().sum(axis=1)>0, f4_col].isna().sum(axis=1).value_counts().sum())[1:]

c_scores = [np.zeros(epochs) for i in range(len(validation_masks))]
f_scores = np.zeros(epochs)

# Training loop
model.train()
for epoch in range(epochs):
    print(f"Epoch {epoch+1} LR {optimizer.param_groups[0]['lr']}")
    
    np.random.shuffle(index)
    losses = 0
    norm_losses = 0
    for i in tqdm(range(n_batches)):
        batch_idx = index[i*batch_size:(i+1)*batch_size]
        # Create batch train data
        srce_mask = torch.tensor(srce_nan_train[batch_idx].astype(np.float32)).to(device)
        x = torch.tensor(X_mixed[batch_idx].astype(np.float32)).to(device)
        mask_init = torch.tensor(random_mask(x.shape, binomial_p=0.05).astype(np.float32)).to(device)
        mask = mask_init - srce_mask * mask_init
        x_masked = x * mask

        # Forward and backward pass
        optimizer.zero_grad()
        p = model(x_masked, mask)
        loss = loss_fn(p, x, srce_mask)
        loss.backward()
        optimizer.step()
        
        losses += loss # Check
    scheduler.step()
        
        
    # Validation stepb
    if (epoch + 1) % valid_per == 0:
        scores = []
        for i in range(len(validation_masks)):
            v = validate(model, validation_masks[i])
            scores.append(v)
            c_scores[i][epoch] = v
            
        final_score = math.sqrt(sum([scores[i]**2 * validation_prob[i] for i in range(len(scores))]))
        f_scores[epoch] = final_score
        
        for i in range(len(scores)):
            print(f'RMSE ({i+1} rows) {scores[i]}')
        print(f'RMSE (TDGP) {final_score}')

TypeError: mask_n_rows() missing 1 required positional argument: 'n_missing'